# 最初のアシスタントを設定する

## アシスタントを作成する

### カーネルの選択を実施

<p>notebook で使うカーネルを選択します</p>

1. カーネルの選択を押下
   ![alt text](./docs/images/choice-kernel1.png)

1. virtualenv で作成した、`extension-lectures`配下の`extension-lectures\Scripts\python.exe`を選択する
   ![alt text](./docs/images/choice-kernel2.png)

1. カーネルの設定ができていることを確認する
   ![alt text](./docs/images/choice-kernel3.png)


### Azure OpenAI のクライアントを作成する


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(verbose=True)


client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

#### アシスタントの作成


In [ ]:
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data."
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model=f"{os.getenv("AZURE_OPENAI_MODEL_NAME")}",  # You must replace this value with the deployment name for your model.
)

#### アシスタントの内容の出力


In [ ]:
print(assistant.model_dump_json(indent=2))

#### スレッドを作成してみます


In [ ]:
thread = client.beta.threads.create()
print(thread)

#### スレッドに追加する最初のユーザーの質問を作成


In [ ]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Create a visualization of a sinewave"
)

#### スレッド メッセージを一覧表示する


In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

#### スレッドを実行する


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

#### スレッド状態を取得する


In [ ]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

status = run.status
print(status)

#### 都度実行では大変なので、ループで状態監視をします


In [ ]:
def check_status(thread, run):
    import time
    from IPython.display import clear_output

    start_time = time.time()

    status = run.status

    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        print(
            "Elapsed time: {} minutes {} seconds".format(
                int((time.time() - start_time) // 60),
                int((time.time() - start_time) % 60),
            )
        )
        status = run.status
        print(f"Status: {status}")
        clear_output(wait=True)

    messages = client.beta.threads.messages.list(thread_id=thread.id)

    print(f"Status: {status}")
    print(
        "Elapsed time: {} minutes {} seconds".format(
            int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)
        )
    )
    print(messages.model_dump_json(indent=2))

In [ ]:
check_status(thread=thread, run=run)

#### 実行後のスレッド メッセージを一覧表示する


In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

print(messages.model_dump_json(indent=2))

#### ファイル ID を取得する

モデルに正弦波の画像を生成させるように要求しました。 画像をダウンロードするには、まず画像ファイル ID を取得する必要があります。


In [ ]:
data = json.loads(
    messages.model_dump_json(indent=2)
)  # Load JSON data into a Python object
image_file_id = data["data"][0]["content"][0]["image_file"]["file_id"]

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

#### イメージをダウンロードする


In [ ]:
content = client.files.content(image_file_id)

image = content.write_to_file("sinewave.png")

<p>ダウンロードしたら、画像をローカルで開きます。</p>


In [ ]:
from PIL import Image
from IPython.display import display

# Display the image in the default image viewer
image = Image.open("sinewave.png")
# image.show()
display(image)

### スレッドでフォローアップの質問をする

アシスタントがこちらの指示に完全には従わず、応答のテキスト部分に実行されたコードが含まれていなかったため、明示的にその情報を要求してみます。


In [ ]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave",
)

もう一度実行して、スレッドの状態を取得する必要があります。


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    # instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

check_status(thread=thread, run=run)

実行状態が完了に達したら、スレッド内のメッセージを再度一覧表示します。これには、最新の質問への応答が含まれているはずです。


In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

print(messages.model_dump_json(indent=2))

最新の質問に対する応答のみを抽出するには、以下を行います。


In [ ]:
data = json.loads(messages.model_dump_json(indent=2))
code = data["data"][0]["content"][0]["text"]["value"]
print(code)

【演習】正弦波の視覚化を生成するために使ったコードを張り付けて実行してみましょう


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ...

#### ダーク モード

最後の質問をスレッドに追加して、コード インタープリターがグラフをダーク モードに切り替えられるかどうかを確認してみましょう。


In [ ]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization.",
)

# Run the thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

status = run.status
check_status(thread=thread, run=run)

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

print(messages.model_dump_json(indent=2))

新しい画像ファイル ID を抽出し、画像をダウンロードして表示します。


In [ ]:
data = json.loads(
    messages.model_dump_json(indent=2)
)  # Load JSON data into a Python object
image_file_id = data[
    "data"
][
    0
][
    "content"
][
    0
][
    "image_file"
][
    "file_id"
]  # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

content = client.files.content(image_file_id)
image = content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
# image.show()
display(image)

## 次の課題に移動へ

[Azure OpenAI + Logic Apps で天気を取得する(準備編)](./docs/logic-apps.markdown)

[README に戻る](./README.markdown)
